In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    max_tokens=1000,
    temperature=0.1
)

template = PromptTemplate.from_template(
    "{animal}이(가) 체온 조절을 위해 하는 행동은 무엇이 있나요?, 그리고 당신의 이름은 무엇인가요?",
)

prompt = template.format(animal="사막 여우")

chat.predict(prompt)

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 {animal_type} 전문가입니다. 당신은 {language}로만 답합니다."),
    ("ai", "안녕하세요, 저는 {name}입니다."),
    ("human", "{animal}이(가) 체온 조절을 위해 하는 행동은 무엇이 있나요?, 그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(
    animal_type="해양 생물",
    name="지성",
    animal="범고래",
    language="한국어"
)

chat.predict_messages(prompt)

AIMessage(content='범고래는 체온을 유지하기 위해 다양한 행동을 합니다. 예를 들어, 물 속에서 수영하거나 플럭싱(피부 혈관을 수축시켜 열을 보존하는 행동)을 하거나, 태양을 피해 그늘에 숨는 등의 행동을 합니다. 또한, 대부분의 고래들은 체온을 유지하기 위해 신진대사를 조절하고 체온 조절을 돕는 지방층을 가지고 있습니다.\n\n제 이름은 지성입니다.')

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how,  ar e,you      ")

['Hello', 'how', 'ar e', 'you']

In [35]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 리스트 생성기입니다. 당신은 모든 질문을 무조건 짧은 단어로 구성되고 쉼표로 구분 된 리스트로 대답해야 합니다. 그리고 최대 {max_items}개까지만 대답하세요. (예시: 사과, 바나나, 수박)"
        ),
        (
            "human",
            "{question}"
        ),
    ]
)

In [38]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "침펜지에 대해서 알려줘."
})

['작은', '과일', '빨간색']